In [1]:
import numpy as np
import yfinance as yf
from datasets import ForexDataWithWindow
from datetime import datetime
from scipy.spatial import distance
from models import time2vec
import mplfinance as mpf

/Users/ryousukekaga/miniforge3/envs/forex/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = ForexDataWithWindow("./data/USDJPY_H1.csv", header=0, normalize=False, data_order="tohlc", input_duration=31, time_index=True)
data = dataset.data

/Users/ryousukekaga/repositories/ForexML/datasets.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["Time"] = pd.to_datetime(self.data["Time"])


In [3]:
model = time2vec.Time2Vec.load_from_checkpoint("usdjpy_model.ckpt", map_location="cpu")

In [4]:
f = np.load("time2vec_udsjpy.npz")

In [11]:
files, vectors, labels = f["arr_0"], f["arr_1"], f["arr_2"]

In [12]:
files = np.array([datetime.strptime(f, "%Y-%m-%d %H_%M_%S") for f in files])

In [21]:
np.where(data.iloc[0].name == files)[0][0]

0

In [7]:
ticker = yf.Ticker("AUDJPY=X")
data_now = ticker.history(interval="15m", period="1d")
vec_now = time2vec.time2vec_from_raw(data_now[-30:], model)

<Figure size 640x480 with 0 Axes>

In [8]:
distances = np.squeeze([distance.cdist(vec_now, [x], "cosine") for x in vectors])

In [ ]:
sim_i = distances.argmax()

In [ ]:
similar_df = data.loc[files[sim_i:sim_i+30]]

In [ ]:
mpf.plot(data_now[-30:], type='candle', returnfig=True, scale_padding=0, style='charles')
mpf.plot(similar_df, type='candle', returnfig=True, scale_padding=0, style='charles')